In [4]:
from astroquery.gaia import Gaia
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')
print(gaiadr3_table)
for column in gaiadr3_table.columns:
  print(column.name)

TAP Table name: gaiadr3.gaia_source
Description: This table has an entry for every Gaia observed source as published with this data release. It contains the basic source parameters, in their final state as processed by the Gaia Data Processing and Analysis Consortium from the raw data coming from the spacecraft. The table is complemented with others containing information specific to certain kinds of objects (e.g.~Solar--system objects, non--single stars, variables etc.) and value--added processing (e.g.~astrophysical parameters etc.). Further array data types (spectra, epoch measurements) are presented separately via Datalink resources.
Size (bytes): 3646930329600
Num. columns: 152
solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pm

In [18]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.gaia import Gaia

Gaia.ROW_LIMIT = 50000  # Ensure the default row limit.
coord = SkyCoord(ra=280, dec=-60, unit=(u.degree, u.degree), frame='icrs')
j = Gaia.cone_search_async(coord, radius=u.Quantity(1.0, u.deg), columns=("source_id", "ra", "dec", "phot_g_mean_mag"))
r = j.get_results()
r.pprint()  

KeyboardInterrupt: 

In [34]:
from astroquery.vizier import Vizier

v = Vizier(columns=['_RAJ2000', '_DEJ2000','B-V', 'Vmag', 'Plx'],
           column_filters={"Vmag":">10"}, keywords=["optical", "xry"])  

gaia_catalog_list = Vizier.find_catalogs('GAIA DR3') 
print({k:v.description for k,v in gaia_catalog_list.items()})
coordinates = SkyCoord(135.9, -65.3, unit=("deg", "deg"))

result = Vizier(catalog='I/355/spectra').query_region(coordinates, radius="1d0m")

#result = result.to_pandas()


{'I/324': 'The Initial Gaia Source List (IGSL) (Smart, 2013)', 'I/337': 'Gaia DR1 (Gaia Collaboration, 2016)', 'I/345': 'Gaia DR2 (Gaia Collaboration, 2018)', 'I/347': 'Distances to 1.33 billion stars in Gaia DR2 (Bailer-Jones+, 2018)', 'I/350': 'Gaia EDR3 (Gaia Collaboration, 2020)', 'I/352': 'Distances to 1.47 billion stars in Gaia EDR3 (Bailer-Jones+, 2021)', 'I/355': 'Gaia DR3 Part 1. Main source (Gaia Collaboration, 2022)', 'I/356': 'Gaia DR3 Part 2. Extra-galactic (Gaia Collaboration, 2022)', 'I/357': 'Gaia DR3 Part 3. Non-single stars (Gaia Collaboration, 2022)', 'I/358': 'Gaia DR3 Part 4. Variability (Gaia Collaboration, 2022)', 'I/359': 'Gaia DR3 Part 5. Solar System (Gaia Collaboration, 2022)', 'I/360': 'Gaia DR3 Part 6. Performance verification (Gaia Collaboration, 2022)', 'I/361': 'Gaia Focused Product Release (Gaia FPR) (Gaia Collaboration, 2023)', 'IV/36': 'Gaia-IPHAS/KIS Value-Added Catalogues (Scaringi+, 2018)', 'VI/137': 'GaiaSimu Universe Model Snapshot (Robin+, 2012)

C:\Users\jcwin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\astroquery\vizier\core.py:908: UserWarning: xry : No such keyword
  warnings.warn("{val} : No such keyword".format(val=val))


AttributeError: 'TableList' object has no attribute 'to_pandas'

In [32]:
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u
import numpy as np
import pyvo as vo

In [11]:
pdr = r.to_pandas()